# Advantage Actor-Critic (A2C)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import numpy as np
import pandas as pd
import flappy_bird_gymnasium
import matplotlib.pyplot as plt
import random
import gymnasium as gym

## Defining Policy Network

In [73]:
class ActorCritic(nn.Module):
    def __init__(self, input_dim=12, hidden_dim=128*2, action_dim=2):
        super(ActorCritic, self).__init__()
        # Shared layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        
        # Policy head (actor)
        self.policy = nn.Linear(hidden_dim, action_dim)
        
        # Value head (critic)
        self.value = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        # x = F.relu(self.fc4(x))
        policy_logits = self.policy(x)
        value = self.value(x)
        return policy_logits, value


## Select Action & Save Log Prob

In [40]:
from torch.distributions import Categorical

def select_action(model, state):
    state = torch.FloatTensor(state).unsqueeze(0)  # shape (1, 12)
    logits, value = model(state)
    probs = F.softmax(logits, dim=-1)
    dist = Categorical(probs)
    action = dist.sample()
    return action.item(), dist.log_prob(action), value


## Training Loop

In [41]:
def train_a2c(env, model, optimizer, gamma=0.99, max_episodes=1000):
    total_reward = 0

    for episode in range(max_episodes):
        state = env.reset()[0]
        log_probs = []
        values = []
        rewards = []
        done = False

        while not done:
            action, log_prob, value = select_action(model, state)
            next_state, reward, done, _, info = env.step(action)
            
            log_probs.append(log_prob)
            values.append(value)
            rewards.append(reward)
            
            state = next_state
            total_reward += reward

        # Compute returns and advantages
        returns = []
        R = 0 if done else model(torch.FloatTensor(state).unsqueeze(0))[1].item()
        for r in reversed(rewards):
            R = r + gamma * R
            returns.insert(0, R)

        returns = torch.tensor(returns)
        values = torch.cat(values)
        log_probs = torch.stack(log_probs)

        advantage = returns - values.squeeze()

        # Losses
        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()
        loss = actor_loss + 0.5 * critic_loss

        # Optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if episode % 100 == 0:
            print(f"Episode {episode} and past 100 episodes, Avg Reward: {total_reward/100:.2f}")
            total_reward = 0


## Training

In [74]:
env = gym.make("FlappyBird-v0", render_mode=None, use_lidar=False)

model = ActorCritic()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_a2c(env, model, optimizer, max_episodes=3000)

# Save model after training
torch.save(model.state_dict(), "saved_policies/AC2_model.pth")

Episode 0 and past 100 episodes, Avg Reward: -0.07


KeyboardInterrupt: 

## Let bot play the game

In [84]:
env = gym.make("FlappyBird-v0", render_mode="human", use_lidar=False)

# Load model
model = ActorCritic()
model.load_state_dict(torch.load("saved_policies/AC2_model_google_final.pth", map_location=torch.device('cpu')))
model.eval()


# Use the trained model
obs, _ = env.reset()

done = False
while not done:

    action, log_prob, value = select_action(model, obs)
    print(action)

    obs, reward, done, truncated, info = env.step(action)
    # print(f"Action {action} gave {reward} reward...")
    # print(obs, reward, done, truncated, info)

env.close()


0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0


In [85]:
env = gym.make("FlappyBird-v0", render_mode=None, use_lidar=False)

import statistics

num_episodes = 1000
episode_rewards = []


# Load model
model = ActorCritic()
model.load_state_dict(torch.load("saved_policies/AC2_model_google_final.pth", map_location=torch.device('cpu')))
model.eval()


for episode in range(num_episodes):
    print(episode)
    # Use the trained model
    obs, _ = env.reset()
    cumulative_reward = 0.0

    done = False
    while not done:

        action, log_prob, value = select_action(model, obs)

        obs, reward, done, truncated, info = env.step(action)
        cumulative_reward += reward
        # print(f"Action {action} gave {reward} reward...")
        # print(obs, reward, done, truncated, info)
    episode_rewards.append(cumulative_reward)

mean_reward = statistics.mean(episode_rewards)
variance_reward = statistics.variance(episode_rewards)

print(f"\nMean Reward over {num_episodes} episodes: {mean_reward}")
print(f"Variance of Reward over {num_episodes} episodes: {variance_reward}")


env.close()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27